In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import nonlinear_QM_FPJPA as Fs
import wx_programs as ex 
import bnc 
from generator import *
from hardware_config import *
from values import * 
from qubit_class import *

In [9]:
pi = np.pi
save_dir = fr"{main_directory}\Python Scripts\Important Blue Fridge Python Files\New\nonlinear_QM\data"
# Example instantiation
q1 = Qubit(
    qubit_id="q1",
    ro_freq=6.7275,
    ge_time=51,
    ef_time=77,
    ge_ssm=-0.111,
    ef_ssm=-0.2525
)

q2 = Qubit(
    qubit_id="q2",
    ro_freq=6.65555,
    ge_time=40,
    ef_time=0,  # Set 0 if not provided in the dictionary
    ge_ssm=-0.152,
    ef_ssm=-0.224
)

print(q1)
print(q2)

Qubit(q1, ro_freq=6.7275, ge_time=51, ef_time=77, ge_ssm=-0.111, ef_ssm=-0.2525)
Qubit(q2, ro_freq=6.65555, ge_time=40, ef_time=0, ge_ssm=-0.152, ef_ssm=-0.224)


In [10]:
q1_dict
q3 = Qubit(
    qubit_id="q3",
    ro_freq = q1_dict['ROq1'],
    ge_time = q1_dict['pi_ge_time_q1']
    
)

{'ROq1': 6.7275,
 'ssm_geq1': -0.111,
 'ssm_efq1': -0.2525,
 'pi_ge_time_q1': 51,
 'pi_ef_time_q1': 77}